In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


from shutil import which
import time
import os
import pandas as pd
import numpy as np
import re


In [2]:
def extract(page_source, start_sep, end_sep):
    source=page_source
#     start_sep='DIMENSIONS'
#     end_sep='MATERIAL'
    result=[]
    tmp=source.split(start_sep)
    for par in tmp:
        if end_sep in par:
            result.append(par.split(end_sep)[0])
#     result = result[3].replace('\\n', '\n')
#     result = "".join([s for s in result.strip().splitlines(True) if s.strip()])
    return result

def extract2(string, start, end):
    result = re.search('%s(.*)%s' % (start, end), string).group(1)
    return result

In [3]:
import requests
url = 'https://www.qeeboo.com/en/products/giraffe-in-love-m-indoor-design-marcantonio?variant=39681840218286'
response_product = requests.get(url)
page_source_product = response_product.text
extract2(page_source_product, '"sku":"', '","offers":')

'19003WH-Z'

In [92]:
chrome_options = Options()
headless = False

if headless:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")


prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
chrome_path = r"C:\Users\chaitanya\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
global driver
driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
driver.get("https://www.qeeboo.com/en/collections/xl-products")
# scroll(1.5)
time.sleep(1)
try:
    cookies_path  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//a[@id = 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']")))
    cookies_path.click()
except:
    pass
product_url = driver.find_elements(By.XPATH, "//div[@class = 'grGrd']/a")
# items = []

for product in product_url:
    print(product.get_attribute('href'))
    
    href = product.get_attribute('href')
    product_details = {}
    product_details['url'] = href
    
    try:
        hero_image_path = product.find_element(By.XPATH, ".//div/div/img")
        hero_image = hero_image_path.get_attribute('data-src')
        
        product_details['hero_image'] = hero_image
    except:
        product_details['hero_image'] = None
    
    try:
        name_path = product.find_element(By.XPATH, ".//div/div[@class = 'fCfgF']/h3")
        product_details['name'] = name_path.text
    except:
        product_details['name'] = None
        
    driver.execute_script("window.open('" +href +"');")
    # switch to new window
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    images_gal = []

    
    varients = driver.find_elements(By.XPATH, "//label[@class = 'Nrrse']")
    if len(varients) > 1:
#     try:
        images_full = []
        varients = driver.find_elements(By.XPATH, "//label[@class = 'Nrrse']")
        for var in varients:
            driver.execute_script(f"window.scrollTo(0,10)")
            var.click()
            time.sleep(1)
            url = driver.current_url
        #             response_product = requests.get(url)
        #             page_source_product = response_product.text
        #             sku = extract2(page_source_product, '"sku":"', '","offers":')
            sku_path = driver.find_elements(By.XPATH, "//script[@type = 'application/ld+json']")[1]

            sku = sku_path.get_attribute('innerHTML')
            sku = extract2(sku, '"sku":"', '","offers":')
            print('sku fucked up \n \n')
            product_details['sku'] = sku

            color = driver.find_elements(By.XPATH, "//div[@class = 'hidden lg:block text-xs']")[0]
            product_details['option(color)'] = color.text
            images = []
            image_path = driver.find_elements(By.XPATH, "//div[@class= 'w-full h-full c6ZxN']/img")

            for img in image_path:
                images.append(img.get_attribute('data-src'))

#             images = [img for img in images if '.gif' not in img]
            varient_image = images[0]
            product_details['variant image'] = varient_image
            images_full.extend(images) 
            print('*'*10)
#             images_full = [img for img in images_full if '.gif' not in img]
            product_details['images'] = images_full
    
            driver.execute_script(f"window.scrollTo(0,1000)")
            image_path_gal = driver.find_elements(By.XPATH, "//div[@class = 'keen-slider UTjST']/div/img")
            for img in image_path_gal:
                images_gal.append(img.get_attribute('id'))

            product_details['images_gal'] = images_gal
            print(product_details)
            items.append(product_details.copy())
            print('varients exists')
    else:
#         try:
        time.sleep(1)
        sku_path = driver.find_elements(By.XPATH, "//script[@type = 'application/ld+json']")[1]

        sku = sku_path.get_attribute('innerHTML')
        sku = extract2(sku, '"sku":"', '","offers":')
        print('sku fucked up \n \n')
        product_details['sku'] = sku
        
        images = []
        image_path = driver.find_elements(By.XPATH, "//div[@class= 'w-full h-full c6ZxN']/img")
        print(image_path)
        for img in image_path:
            images.append(img.get_attribute('data-src'))
        print(images)
#         images = [img for img in images if '.gif' not in img]
        product_details['images'] = images
        driver.execute_script(f"window.scrollTo(0,1000)")
        image_path_gal = driver.find_elements(By.XPATH, "//div[@class = 'keen-slider UTjST']/div/img")
        for img in image_path_gal:
            images_gal.append(img.get_attribute('id'))

        product_details['images_gal'] = images_gal 
        print(product_details)

        items.append(product_details)
        print('#'*10)
#         except:
#             print(product_details)
#             items.append(product_details)
#             print('$'*10)
#             pass

    #     try:
    
        
#     except:
#         pass

    driver.close()
    #back to main window
    driver.switch_to.window(driver.window_handles[0])

C:\Users\CHAITA~1\AppData\Local\Temp/ipykernel_4704/172772460.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


https://www.qeeboo.com/en/products/fallen-chandelier-xl-design-studio-job?variant=32376902418478
sku fucked up 
 

**********
{'url': 'https://www.qeeboo.com/en/products/fallen-chandelier-xl-design-studio-job?variant=32376902418478', 'hero_image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Fallen-Chandelier-XL--design-Studio-Job--01--black.png?v=1640771126', 'name': 'Fallen Chandelier XL', 'sku': '37002PI', 'option(color)': 'Black', 'variant image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Fallen-Chandelier-XL--design-Studio-Job--01--black.png?v=1640771110', 'images': ['https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Fallen-Chandelier-XL--design-Studio-Job--01--black.png?v=1640771110', 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Fallen-Chandelier-XL--design-Studio-Job--02--black.png?v=1640771113', 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Fallen-Chandelier-XL--design-Studio-Job--03--black.png?v=1

https://www.qeeboo.com/en/products/paris-xl-design-studio-job?variant=31540907606062
sku fucked up 
 

**********
{'url': 'https://www.qeeboo.com/en/products/paris-xl-design-studio-job?variant=31540907606062', 'hero_image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Paris-XL--design-Studio-Job--01--black.png?v=1640770556', 'name': 'Paris XL', 'sku': '32001BL', 'option(color)': 'Black', 'variant image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Paris-XL--design-Studio-Job--01--black.png?v=1640770534', 'images': ['https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Paris-XL--design-Studio-Job--01--black.png?v=1640770534', 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Paris-XL--design-Studio-Job--02--black.png?v=1640770538', 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Paris-XL--design-Studio-Job--03--black.png?v=1640770542'], 'images_gal': ['https://inside.qeeboo.com/_media/?src=https://inside.qeeboo.com/_me

sku fucked up 
 

**********
{'url': 'https://www.qeeboo.com/en/products/giraffe-in-love-xl-outdoor-design-marcantonio?variant=31112881831982', 'hero_image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-outdoor-4m-white-design-marcantonio-qeeboo--01a--white.png?v=1620980859', 'name': 'Giraffe in Love XL Outdoor', 'sku': '19001BL', 'option(color)': 'Black', 'variant image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-outdoor-4m-black-design-marcantonio-qeeboo--02a--black.png?v=1620980873', 'images': ['https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-outdoor-4m-white-design-marcantonio-qeeboo--01a--white.png?v=1620980859', 'https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-outdoor-4m-white-design-marcantonio-qeeboo--01b--white.png?v=1620980864', 'https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-outdoor-4m-white-design-marcantonio-qeeboo--01c--white.png?v=1620980868', 'https://cdn

In [48]:
items

[{'url': 'https://www.qeeboo.com/en/products/giraffe-in-love-m-indoor-design-marcantonio?variant=39681840218286',
  'hero_image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--New-Giraffe-in-Love-M-Indoor--design-Marcantonio--01--white.png?v=1639414027',
  'name': 'Giraffe in Love M Indoor',
  'sku': '19003WH-Z',
  'option(color)': 'White',
  'variant image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--New-Giraffe-in-Love-M-Indoor--design-Marcantonio--01--white.png?v=1639414027',
  'images': ['https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--New-Giraffe-in-Love-M-Indoor--design-Marcantonio--01--white.png?v=1639414027',
   'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--New-Giraffe-in-Love-M-Indoor--design-Marcantonio--02--white.png?v=1639414027',
   'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--New-Giraffe-in-Love-M-Indoor--design-Marcantonio--03--white.png?v=1639414027',
   'https://cdn.shopify.com/s/files/1/2

In [93]:
len(items)

498

In [94]:
product_details

{'url': 'https://www.qeeboo.com/en/products/giraffe-in-love-xl-indoor-design-marcantonio?variant=16857678676014',
 'hero_image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Giraffe-in-Love-Indoor--design-Marcantonio--01--white.png?v=1639414012',
 'name': 'Giraffe in Love XL Indoor',
 'sku': '19002BL',
 'option(color)': 'Black',
 'variant image': 'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Giraffe-in-Love-Indoor--design-Marcantonio--04--black.png?v=1639414012',
 'images': ['https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Giraffe-in-Love-Indoor--design-Marcantonio--01--white.png?v=1639414012',
  'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Giraffe-in-Love-Indoor--design-Marcantonio--02--white.png?v=1639414012',
  'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Giraffe-in-Love-Indoor--design-Marcantonio--03--white.png?v=1639414012',
  'https://cdn.shopify.com/s/files/1/2283/1973/products/Qeeboo--Giraffe-in-Lov

In [101]:
import pandas as pd
import numpy as np

df = pd.DataFrame(items, )
df


,url,hero_image,name,sku,option(color),variant image,images,images_gal
0,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Indoor,19003WH-Z,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[]
1,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Indoor,19003BL-Z,Black,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[]
2,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Outdoor,19004WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[]
3,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Outdoor,19004BL,Black,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[]
4,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love Wall Lamp,29003WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[https://inside.qeeboo.com/_media/?src=https:/...
...,...,...,...,...,...,...,...,...
493,https://www.qeeboo.com/en/products/paris-xl-de...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Paris XL,32001WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[https://inside.qeeboo.com/_media/?src=https:/...
494,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love XL Outdoor,19001WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[https://inside.qeeboo.com/_media/?src=https:/...
495,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love XL Outdoor,19001BL,Black,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[https://inside.qeeboo.com/_media/?src=https:/...
496,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love XL Indoor,19002WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://cdn.shopify.com/s/files/1/2283/1973/p...,[https://inside.qeeboo.com/_media/?src=https:/...


In [102]:
df = df.astype(object).replace(np.nan, 'None')


In [103]:
for i in range(len(df)):
    df['images'][i] = df['images'][i] + df['images_gal'][i]
    
    try:
        df['images'][i] = [img for img in df['images'][i] if '.gif' not in img]
    except:
        pass
    try:
        if df['hero_image'][i] is None:
            df['hero_image'][i] = df['images'][i][0]
    except:
        pass
    try:
        if df['variant image'][i] is None:
            df['variant image'][i] = df['images'][i][0]
    except:
        pass
    try:
        df['images'][i] = '\n'.join(df['images'][i])
    except:
        pass
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [104]:
df

,url,hero_image,name,sku,option(color),variant image,images,images_gal
0,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Indoor,19003WH-Z,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[]
1,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Indoor,19003BL-Z,Black,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[]
2,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Outdoor,19004WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[]
3,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love M Outdoor,19004BL,Black,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[]
4,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love Wall Lamp,29003WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://inside.qeeboo.com/_media/?src=https:/...
...,...,...,...,...,...,...,...,...
493,https://www.qeeboo.com/en/products/paris-xl-de...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Paris XL,32001WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://inside.qeeboo.com/_media/?src=https:/...
494,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love XL Outdoor,19001WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://inside.qeeboo.com/_media/?src=https:/...
495,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love XL Outdoor,19001BL,Black,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://inside.qeeboo.com/_media/?src=https:/...
496,https://www.qeeboo.com/en/products/giraffe-in-...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,Giraffe in Love XL Indoor,19002WH,White,https://cdn.shopify.com/s/files/1/2283/1973/pr...,https://cdn.shopify.com/s/files/1/2283/1973/pr...,[https://inside.qeeboo.com/_media/?src=https:/...


In [105]:
df.to_excel('qeeboo_full2.xlsx' )

C:\Users\chaitanya\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1168: UserWarning: Ignoring URL 'https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-wall-lamp-white-design-marcantonio-qeeboo--01a--white.png?v=1639414002
https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-wall-lamp-white-design-marcantonio-qeeboo--01b--white.png?v=1639414002
https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-wall-lamp-white-design-marcantonio-qeeboo--01c--white.png?v=1639414002
https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-wall-lamp-black-design-marcantonio-qeeboo--02a--black.png?v=1639414002
https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-wall-lamp-black-design-marcantonio-qeeboo--02b--black.png?v=1619512211
https://cdn.shopify.com/s/files/1/2283/1973/products/giraffe-in-love-wall-lamp-black-design-marcantonio-qeeboo--02c--black.png?v=1619512211
https://inside.qeeboo.com/_media/?src=https://inside.qee